SETTING UP FLASK SERVER TO RECIEVE JSON DATA AND CONVERTING TO PROMETHEUS METRICS

In [1]:
# Importing necessary libraries
from flask import Flask, request, jsonify
from flask_cors import CORS
from prometheus_client import start_http_server, Gauge
from prometheus_flask_exporter import PrometheusMetrics
import json
import time
from threading import Thread
import requests

In [2]:
# Defining Flask server
app = Flask(__name__)
CORS(app)

# Initialize Prometheus metrics
metrics = PrometheusMetrics(app)

In [3]:
# Define a Prometheus Gauge metric
acceleration_metric = Gauge('total_acceleration', 'Total Acceleration Metric', ['axis'])
accelerometer_metric = Gauge('accelerometer', 'Accelerometer Metric', ['axis'])

In [4]:
# Processing json data
def process_acceleration_data(data):
    for entry in data["payload"]:
        if entry["name"] == "totalacceleration":
            for axis, value in entry["values"].items():
                acceleration_metric.labels(axis=axis).set(value)

In [5]:
# Posting reqiest to the url
def fetch_and_update_metrics():
    while True:
        try:
            json_data = requests.post('http://192.168.190.213:6000/json_endpoint').json()

        except Exception as e:
            pass

        time.sleep(2) # Adjusting the sleep interval 

In [6]:
# Recieving JSON data
@app.route('/json_endpoint', methods=['POST'])
def receive_json():
    try:
        data = request.get_json()

        print(data) # Printing JSON data
        
        process_acceleration_data(data)    #Processing JSON data
        process_accelerometer_data(data)
            
        a=random.randint(60,100)           # Heartbeat metric
        heart_beat.labels(axis="heart_beat").set(a)

    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)})

In [7]:
# Final set up
if __name__ == '__main__':
    # Start the Flask application
    flask_thread = Thread(target=app.run, kwargs={'host': '192.168.190.213', 'port': 6000})
    flask_thread.start()

    # Start the Prometheus HTTP server on port 8000
    start_http_server(8000)

    # Start the thread for fetching and updating metrics
    metrics_thread = Thread(target=fetch_and_update_metrics)
    metrics_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.190.213:6000
Press CTRL+C to quit
192.168.190.213 - - [04/Mar/2024 18:59:13] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:15] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.135 - - [04/Mar/2024 18:59:15] "POST /json_endpoint HTTP/1.1" 200 -


{'messageId': 4, 'sessionId': 'e788f04f-2183-4c07-a7f1-df6e95fa321d', 'deviceId': '2274ee0c-3545-4831-b434-15dbe46693c4', 'payload': [{'name': 'accelerometer', 'values': {'z': 0.47371530532836914, 'y': 0.2483515739440918, 'x': -0.31769800186157227}, 'accuracy': 3, 'time': 1709558953757436400}, {'name': 'accelerometeruncalibrated', 'values': {'z': 7.768298625946045, 'y': 5.828916072845459, 'x': 3.0993034839630127}, 'accuracy': 3, 'time': 1709558953803825200}, {'name': 'accelerometer', 'values': {'z': 0.36726903915405273, 'y': -0.11674022674560547, 'x': 0.15949082374572754}, 'accuracy': 3, 'time': 1709558953807436300}, {'name': 'totalacceleration', 'values': {'z': 6.523432731628418, 'y': 6.760322093963623, 'x': 2.730210542678833}, 'accuracy': 3, 'time': 1709558953844102000}, {'name': 'accelerometeruncalibrated', 'values': {'z': 5.084748268127441, 'y': 6.2913289070129395, 'x': 2.6153552532196045}, 'accuracy': 3, 'time': 1709558953884575000}, {'name': 'accelerometer', 'values': {'z': 0.207

192.168.190.135 - - [04/Mar/2024 18:59:16] "POST /json_endpoint HTTP/1.1" 200 -


{'messageId': 5, 'sessionId': 'e788f04f-2183-4c07-a7f1-df6e95fa321d', 'deviceId': '2274ee0c-3545-4831-b434-15dbe46693c4', 'payload': [{'name': 'accelerometer', 'values': {'z': 29.507556915283203, 'y': -3.3802781105041504, 'x': -10.999567031860352}, 'accuracy': 3, 'time': 1709558954707436300}, {'name': 'accelerometeruncalibrated', 'values': {'z': 12.640684127807617, 'y': 3.3104701042175293, 'x': -2.472383975982666}, 'accuracy': 3, 'time': 1709558954770470700}, {'name': 'accelerometer', 'values': {'z': 20.381996154785156, 'y': -2.818694591522217, 'x': -7.424241542816162}, 'accuracy': 3, 'time': 1709558954757436400}, {'name': 'totalacceleration', 'values': {'z': 5.154139995574951, 'y': 1.6151553392410278, 'x': -0.9421738982200623}, 'accuracy': 3, 'time': 1709558954810747400}, {'name': 'accelerometer', 'values': {'z': 6.827707290649414, 'y': -4.606217384338379, 'x': -4.616641044616699}, 'accuracy': 3, 'time': 1709558954807436300}, {'name': 'accelerometeruncalibrated', 'values': {'z': 1.780

...

192.168.190.135 - - [04/Mar/2024 18:59:18] "POST /json_endpoint HTTP/1.1" 200 -


{'messageId': 7, 'sessionId': 'e788f04f-2183-4c07-a7f1-df6e95fa321d', 'deviceId': '2274ee0c-3545-4831-b434-15dbe46693c4', 'payload': [{'name': 'accelerometer', 'values': {'z': 15.945456504821777, 'y': -9.109302520751953, 'x': -8.526298522949219}, 'accuracy': 3, 'time': 1709558956757436400}, {'name': 'accelerometeruncalibrated', 'values': {'z': 32.77090072631836, 'y': 6.932605266571045, 'x': -15.762121200561523}, 'accuracy': 3, 'time': 1709558956784319000}, {'name': 'totalacceleration', 'values': {'z': 14.587244987487793, 'y': 3.2410783767700195, 'x': -3.9254255294799805}, 'accuracy': 3, 'time': 1709558956824596200}, {'name': 'accelerometer', 'values': {'z': 31.128646850585938, 'y': 0.039114952087402344, 'x': -17.31966781616211}, 'accuracy': 3, 'time': 1709558956807436300}, {'name': 'accelerometeruncalibrated', 'values': {'z': 5.977869033813477, 'y': 1.1485549211502075, 'x': -1.5864413976669312}, 'accuracy': 3, 'time': 1709558956864678000}, {'name': 'accelerometer', 'values': {'z': 10.7

192.168.190.213 - - [04/Mar/2024 18:59:19] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.135 - - [04/Mar/2024 18:59:19] "POST /json_endpoint HTTP/1.1" 200 -


{'messageId': 8, 'sessionId': 'e788f04f-2183-4c07-a7f1-df6e95fa321d', 'deviceId': '2274ee0c-3545-4831-b434-15dbe46693c4', 'payload': [{'name': 'accelerometer', 'values': {'z': 28.862337112426758, 'y': -6.557877063751221, 'x': -11.905293464660645}, 'accuracy': 3, 'time': 1709558957757436400}, {'name': 'totalacceleration', 'values': {'z': 20.716461181640625, 'y': 3.3678979873657227, 'x': -0.6885347366333008}, 'accuracy': 3, 'time': 1709558957791052000}, {'name': 'accelerometeruncalibrated', 'values': {'z': 8.070392608642578, 'y': 3.8901314735412598, 'x': -2.196611166000366}, 'accuracy': 3, 'time': 1709558957831524400}, {'name': 'accelerometer', 'values': {'z': 25.718090057373047, 'y': -2.257134437561035, 'x': -6.289730072021484}, 'accuracy': 3, 'time': 1709558957807436300}, {'name': 'totalacceleration', 'values': {'z': 2.1230318546295166, 'y': 0.6191428303718567, 'x': 1.7078771591186523}, 'accuracy': 3, 'time': 1709558957871801600}, {'name': 'accelerometer', 'values': {'z': 5.92319822311

192.168.190.213 - - [04/Mar/2024 18:59:21] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:23] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:25] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:27] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:29] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:31] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:33] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:35] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:37] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:39] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:41] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:43] "POST /json_endpoint HTTP/1.1" 200 -
192.168.190.213 - - [04/Mar/2024 18:59:4